## Importing the required libraries

In [ ]:
!git clone -l -s -b master git://github.com/srgynmv/EmoteNN.git emotenn
%cd emotenn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from emotenn import constants as ct, train_utils as tu, load_utils as lu, plot_utils as pu

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import Input, Flatten, Dropout, Activation
from tensorflow.keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from tensorflow.keras.models import Model

## Download and preprocess dataset

In [ ]:
X, Y = lu.load_dataset(ct.RAVDESS)
# Changing dimensions for CNN models
X = np.mean(X, axis=1)
X = np.expand_dims(X, -1)

In [ ]:
split = tu.split_dataset(X, Y, test_size=0.15, valid_size=0.2)
X_train, X_valid, X_test, Y_train, Y_valid, Y_test = split

## Creating and training a model

In [ ]:
strategy = tu.get_distribution_strategy()
with strategy.scope():
    model = Sequential()
    model.add(Conv1D(256, 5,padding='same',
                    input_shape=(216,1)))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5,padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(MaxPooling1D(pool_size=(8)))
    model.add(Conv1D(128, 5,padding='same',))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5,padding='same',))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5,padding='same',))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Conv1D(128, 5,padding='same',))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(7))
    model.add(Activation('softmax'))
    opt = tf.keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

In [ ]:
train_history = model.fit(X_train, 
                          Y_train, 
                          batch_size=16, 
                          epochs=300, 
                          validation_data=(X_valid, Y_valid),
                          shuffle=True)

In [ ]:
score = model.evaluate(X_test, Y_test)
print("Dataset {}: {:.2f}%".format(model.metrics_names[1], score[1] * 100))

In [ ]:
pu.plot_model_history(train_history.history)

## Saving the model

In [ ]:
tu.save_results(model, train_history, 'sound_1d_cnn')